In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px


<ipython-input-1-89e3ae8eb04c>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-89e3ae8eb04c>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv(url)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


In [3]:
spacex_df['Launch Site'].unique()

spacex_df.head()


,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [ ]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                 dcc.Dropdown(id='site-dropdown',
                                 options=[
                                 {'label': 'All Sites', 'value': 'ALL'},
                                 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                  ],
                                 value='ALL',
                                 placeholder="place holder here",
                                 searchable=True
                                 ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                               dcc.RangeSlider(id='payload-slider',
                                min=0, max=10000, step=100,
                                marks={0: '0', 2500: '2500',5000:'5000', 7500:'7500', 10000:'10000'},
                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        df =spacex_df.value_counts('class').reset_index(name='n')
        fig = px.pie(df, values = 'n', 
        names='class', 
        title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site].value_counts('class').reset_index(name='class count')
        title = f"Total Success Launches for site {entered_site}"
        fig = px.pie(filtered_df, values = 'class count',
        names = 'class', title = title)
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def get_pie_chart(entered_site,payload):
    low, high=payload
    if entered_site == 'ALL':
        df_slide=spacex_df[(spacex_df['Payload Mass (kg)']>=low)&(spacex_df['Payload Mass (kg)']<=high)]
        fig = px.scatter(df_slide, x='Payload Mass (kg)',y='class',color='Booster Version Category',
        title='Total Success Launches By Site')
        return fig
    else:
        df_filter = spacex_df[spacex_df['Launch Site'== entered_site]]
        df_filter = df_filter[(spacex_df['Payload Mass (kg)']>=low)&(spacex_df['Payload Mass (kg)']<=high)]  
        fig = px.scatter(df_filter, x='Payload Mass (kg)',
        y='class',color='Booster Version Category',
        title='Total Success Launches By Site')
        return fig
        
    



# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Mar/2022 17:20:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2022 17:20:09] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2022 17:20:09] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2022 17:20:09] "GET /_favicon.ico?v=2.2.0 HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2022 17:20:09] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2022 17:20:09] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2022 17:20:09] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2022 17:20:09] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2022 17:20:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2022 17:20:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2022 17:20:28] "POST /_dash-upd

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\baghe\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: False

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\baghe\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\baghe\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle

127.0.0.1 - - [12/Mar/2022 17:20:28] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [12/Mar/2022 17:21:47] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\baghe\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: False

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\baghe\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\baghe\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle

...

127.0.0.1 - - [12/Mar/2022 17:33:39] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\baghe\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: False

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\baghe\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\baghe\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle

127.0.0.1 - - [12/Mar/2022 17:33:43] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\baghe\anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: False

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\baghe\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\baghe\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle

127.0.0.1 - - [12/Mar/2022 17:33:46] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [12/Mar/2022 17:33:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2022 17:33:58] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2022 17:34:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Mar/2022 17:34:07] "POST /_dash-update-component HTTP/1.1" 200 -


In [ ]:
 spacex_df['Launch Site'].unique()
value ='CCAFS LC-40'

In [ ]:
filtered_df = spacex_df[spacex_df['Launch Site']==value]


In [ ]:
df =spacex_df.value_counts('class').rename_axis('class').reset_index(name='n')

df 

In [ ]:
df =spacex_df.value_counts('class').reset_index(name='n')
df

max_payload

In [ ]:
spacex_df.keys()
